# Build Dataset

## Ideas

- Stadium
- Weather
- Historical
    - Last x games
    - Last x days
    - Career
    - Career vs pitcher (hitter)
- Team stats
    - Opp hitting
    - Bullpen pitching
    
Game Directory

- Home Team
- Away Team
- Starting Pitcher Stats
    - general prefermance
    - against hitter
- Team Stats
    - Defensive Stats??
    - W/L??
    - Team Batting Stats??
- Month


Cleanse Dataset

- Fix AB issue
    - remove bats without stats (ABs)
- BO zero thing - remove
- 1v2v3B
    - 3 is categorical for some reason
    

## Batter Exploration

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
pd.set_option('display.max_columns', 500)

In [10]:
batter_raw = pd.read_csv(r'..\Data_Pull\data_files\reduced_data\batter_raw_red.csv', index_col='Unnamed: 0', parse_dates = ['Date'])

In [11]:
len(batter_raw)

166885

In [12]:
def gen_avg(df, stat, time_period, agg_type):
    
    """
    Function to average previous x amount of lines.
    Requires dataframe to be sorted by time and player.
    
    Args
    df - dataframe to loop over
    stat - column to average
    time_period - number of rows to average
    agg_type - how to aggregate over time (sum/mean/count)
    """

    ## Create new column
    if time_period < 100000:
        new_col_name = '{0}_{1}'.format(stat, str(time_period))
    else:
        new_col_name = '{0}_lifetime'.format(stat)
    
    ## Aggregate based on specified input
    if agg_type in ['Sum', 'sum']:
        df[new_col_name]=df.groupby('pid')[stat].apply(lambda x : x.shift().rolling(time_period, min_periods = 1).sum())
    
    elif agg_type in ['Mean', 'mean']:
        df[new_col_name]=df.groupby('pid')[stat].apply(lambda x : x.shift().rolling(time_period, min_periods = 1).mean())
        
    elif agg_type in ['Sum', 'sum']:
        df[new_col_name]=df.groupby('pid')[stat].apply(lambda x : x.shift().rolling(time_period, min_periods = 1).count())
    
    else:
        print('Error: Enter Valid Aggregation Type')

In [13]:
batter_raw.sort_values(['pid', 'Date'], inplace = True)
batter_raw.reset_index(inplace = True, drop = True)

In [14]:
batter_raw.head(5)

,Date,Team,Opp,BO,Pos,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,pid
0,2017-04-05,MIL,COL,9,P,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,18.0
1,2017-04-06,MIL,COL,9,P,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,18.0
2,2017-04-07,MIL,CHC,9,P,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,18.0
3,2017-04-11,MIL,@TOR,0,P,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,18.0
4,2017-04-12,MIL,@TOR,0,P,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,18.0


## General Info Parse

In [15]:
gen_inf = pd.read_csv(r'../Data_Pull/data_files/general_info.csv', index_col='Unnamed: 0')

In [16]:
gen_inf.head()

,Age,Bats/Throws,Birthdate,Height/Weight,Name,POS,pid
0,Age: 41,Bats/Throws: B/R,"1/16/1978 (41 y, 4 m, 20 d)","5' 11"" / 165",Alfredo Amezaga,OF,1.0
1,Age: 44,Bats/Throws: R/R,"1/20/1975 (44 y, 4 m, 16 d)","5' 7"" / 177",David Eckstein,SS,10.0
2,Age: 48,Bats/Throws: R/R,"5/11/1971 (48 y, 25 d)","6' 2"" / 220",Kerry Ligtenberg,P,100.0
3,Age: 50,Bats/Throws: R/R,"10/22/1968 (50 y, 7 m, 14 d)","6' 0"" / 200",Keith Osik,C,1001.0
4,Age: 31,Bats/Throws: R/R,"6/18/1987 (31 y, 11 m, 18 d)","6' 5"" / 225",Taylor Thompson,P,10019.0


In [17]:
len(gen_inf)

5070

In [18]:
## Parse columns
gen_inf.loc[:, 'Age'] = gen_inf.loc[:, 'Age'].apply(lambda x: int(x.split(':')[1].strip()))
gen_inf.loc[:, 'Bats'] = gen_inf.loc[:, 'Bats/Throws'].apply(lambda x: x.split('/')[1].split(':')[1].strip())
gen_inf.loc[:, 'Throws'] = gen_inf.loc[:, 'Bats/Throws'].apply(lambda x: x.split('/')[2].strip())
gen_inf.loc[:, 'Birthdate'] = gen_inf.loc[:, 'Birthdate'].apply(lambda x: pd.to_datetime(x.split('(')[0].strip()))
gen_inf.loc[:, 'Height'] = gen_inf.loc[:, 'Height/Weight'].apply(lambda x: (int(x.split("'")[0].strip()) * 12) + int(x.split('"')[0].split("'")[1].strip()))
gen_inf.loc[:, 'Weight'] = gen_inf.loc[:, 'Height/Weight'].apply(lambda x: int(x.split('/')[1].strip()))

## Drop not needed cols
gen_inf.drop(columns = {'Bats/Throws', 'Height/Weight'}, inplace = True)

In [19]:
gen_inf.to_csv(r'../Data_Pull/data_files/general_info_clean.csv')

In [20]:
gen_inf.head()

,Age,Birthdate,Name,POS,pid,Bats,Throws,Height,Weight
0,41,1978-01-16,Alfredo Amezaga,OF,1.0,B,R,71,165
1,44,1975-01-20,David Eckstein,SS,10.0,R,R,67,177
2,48,1971-05-11,Kerry Ligtenberg,P,100.0,R,R,74,220
3,50,1968-10-22,Keith Osik,C,1001.0,R,R,72,200
4,31,1987-06-18,Taylor Thompson,P,10019.0,R,R,77,225


## 2 Game Days Issue

In [21]:
batter_raw = pd.read_csv(r'..\Data_Pull\data_files\reduced_data\batter_raw_red.csv', index_col='Unnamed: 0', parse_dates = ['Date'])
pitching_raw = pd.read_csv(r'..\Data_Pull\data_files\reduced_data\pitching_raw_red.csv', index_col='Unnamed: 0', parse_dates = ['Date'])

In [22]:
pitching_raw.sort_values(['pid', 'Date'], inplace = True)
pitching_raw.reset_index(inplace = True, drop = True)

In [23]:
batter_raw = batter_raw.loc[batter_raw['Date'] >= pd.to_datetime('2000-1-1')]
pitching_raw = pitching_raw.loc[pitching_raw['Date'] >= pd.to_datetime('2000-1-1')]

In [24]:
batter_raw.head(3)

,Date,Team,Opp,BO,Pos,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,pid
0,2017-08-13,KCR,@CHW,0,P,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,18.0
1,2017-08-10,KCR,@STL,9,P,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,18.0
2,2017-08-08,KCR,STL,0,P,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,18.0


In [25]:
pitching_raw.head(3)

,Date,Team,Opp,GS,W,L,SV,HLD,IP,TBF,H,R,ER,HR,BB,SO,K/9,BB/9,HR/9,BABIP,LOB%,GB%,HR/FB,ERA,FIP,xFIP,GSv2,pid
0,2017-04-05,MIL,COL,0,0,0,1,0.0,0.1,1,0,0,0,0,0,1,27.0,0.0,0.0,0.0,1.0,0.000,0.0,0.0,-2.84,-2.84,NaN,18.0
1,2017-04-06,MIL,COL,0,0,1,0,0.0,1.0,4,1,1,1,1,0,1,9.0,0.0,9.0,0.0,1.0,0.667,1.0,9.0,14.16,2.94,NaN,18.0
2,2017-04-07,MIL,CHC,0,0,0,0,0.0,0.1,1,0,0,0,0,0,1,27.0,0.0,0.0,0.0,1.0,0.000,0.0,0.0,-2.84,-2.84,NaN,18.0


In [26]:
batter_raw.loc[:, 'Home/Away'] = batter_raw.loc[:, 'Opp'].apply(lambda x: 'Away' if '@' in x else 'Home')
pitching_raw.loc[:, 'Home/Away'] = pitching_raw.loc[:, 'Opp'].apply(lambda x: 'Away' if '@' in x else 'Home')

batter_raw.loc[:, 'Opp'] = batter_raw.loc[:, 'Opp'].apply(lambda x: x.split('@')[1] if '@' in x else x)
pitching_raw.loc[:, 'Opp'] = pitching_raw.loc[:, 'Opp'].apply(lambda x: x.split('@')[1] if '@' in x else x)

In [27]:
## Function to make sure matchup can be join key (sort alphabetically)
def sort_teams(a, b):
    
    sort_list = [a, b]
    
    sort_list = sorted(sort_list)
    
    name = '_'.join(sort_list)
    
    return name

batter_raw.loc[:, 'Teams'] = batter_raw.loc[:, ['Team', 'Opp']].apply(lambda x: sort_teams(*x), axis = 1)
pitching_raw.loc[:, 'Teams'] = pitching_raw.loc[:, ['Team', 'Opp']].apply(lambda x: sort_teams(*x), axis = 1)

In [28]:
two_game_issue = pitching_raw.loc[pitching_raw['GS'] >= 1].groupby(['Date', 'Teams'])['GS'].count()[pitching_raw[pitching_raw['GS'] >= 1].groupby(['Date', 'Teams'])['GS'].count() > 1].reset_index()
two_game_issue = two_game_issue.loc[two_game_issue['GS'] > 2]

## Generate Historical Stats

In [29]:
adv_df = pd.read_csv(r'..\Data_Pull\data_files\reduced_data\batter_advanced_raw_red.csv', index_col='Unnamed: 0', parse_dates = ['Date'])

In [30]:
adv_df.sort_values(['pid', 'Date'], inplace = True)
adv_df.reset_index(inplace = True, drop = True)

In [31]:
bat_all = pd.merge(batter_raw, adv_df, how = 'outer', on = ['pid', 'Date'])
bat_all = bat_all.loc[:, [x for x in bat_all.columns if '_y' not in x]]

In [32]:
bat_all = bat_all[~pd.isnull(bat_all['Team_x'])]

In [35]:
bat_all.head()

,Date,Team_x,Opp_x,BO_x,Pos_x,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG_x,pid,Home/Away,Teams,BB%,K%,BB/K,OBP,SLG,OPS,ISO,Spd,BABIP,wSB,wRC,wRAA,wOBA,wRC+
117,2018-07-31,TEX,ARI,9.0,P,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,375.0,Away,ARI_TEX,0.0,1.000,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0,-0.5,0.0,-100.0
125,2018-06-12,TEX,LAD,9.0,P,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,375.0,Away,LAD_TEX,0.0,0.500,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0,-0.5,0.0,-100.0
149,2017-08-09,MIN,MIL,9.0,P,1.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,375.0,Away,MIL_MIN,0.0,0.333,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0,-0.8,0.0,-100.0
152,2017-07-24,MIN,LAD,9.0,P,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,375.0,Away,LAD_MIN,0.0,0.500,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0,-0.5,0.0,-100.0
154,2017-06-28,ATL,SDP,9.0,P,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,375.0,Away,ATL_SDP,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0,-0.3,0.0,-100.0


In [34]:
bat_all = bat_all.loc[(bat_all['AB'] != 0) & (bat_all['BO_x'] != 0)]

In [36]:
for col in [x for x in bat_all.columns if bat_all[x].dtype in [np.int64, np.float64] and x != 'pid']:
    for days in [30]: #1, 3, 7, 15, 30, 1000000
        if col == 'BO_x':
            gen_avg(bat_all, col, days, 'mean')
        else:
            gen_avg(bat_all, col, days, 'sum')
        print('{0} {1} Complete'.format(col, str(days)))

BO_x 30 Complete
G 30 Complete
AB 30 Complete
PA 30 Complete
H 30 Complete
1B 30 Complete
2B 30 Complete
3B 30 Complete
HR 30 Complete
R 30 Complete
RBI 30 Complete
BB 30 Complete
IBB 30 Complete
SO 30 Complete
HBP 30 Complete
SF 30 Complete
SH 30 Complete
GDP 30 Complete
SB 30 Complete
CS 30 Complete
AVG_x 30 Complete
BB% 30 Complete
K% 30 Complete
BB/K 30 Complete
OBP 30 Complete
SLG 30 Complete
OPS 30 Complete
ISO 30 Complete
Spd 30 Complete
BABIP 30 Complete
wSB 30 Complete
wRC 30 Complete
wRAA 30 Complete
wOBA 30 Complete
wRC+ 30 Complete


In [178]:
for col in [x for x in batter_raw.columns if batter_raw[x].dtype == np.int64]:
    for days in [1, 7, 30, 1000000]: #1, 3, 7, 15, 30, 1000000
        if col == 'BO':
            gen_avg(batter_raw, col, days, 'mean')
        else:
            gen_avg(batter_raw, col, days, 'sum')
        print('{0} {1} Complete'.format(col, str(days)))

BO 1 Complete
BO 7 Complete
BO 30 Complete
BO 1000000 Complete
G 1 Complete
G 7 Complete
G 30 Complete
G 1000000 Complete
AB 1 Complete
AB 7 Complete
AB 30 Complete
AB 1000000 Complete
PA 1 Complete
PA 7 Complete
PA 30 Complete
PA 1000000 Complete
H 1 Complete
H 7 Complete
H 30 Complete
H 1000000 Complete
1B 1 Complete
1B 7 Complete
1B 30 Complete
1B 1000000 Complete
2B 1 Complete
2B 7 Complete
2B 30 Complete
2B 1000000 Complete
3B 1 Complete
3B 7 Complete
3B 30 Complete
3B 1000000 Complete
HR 1 Complete
HR 7 Complete
HR 30 Complete
HR 1000000 Complete
R 1 Complete
R 7 Complete
R 30 Complete
R 1000000 Complete
RBI 1 Complete
RBI 7 Complete
RBI 30 Complete
RBI 1000000 Complete
BB 1 Complete
BB 7 Complete
BB 30 Complete
BB 1000000 Complete
IBB 1 Complete
IBB 7 Complete
IBB 30 Complete
IBB 1000000 Complete
SO 1 Complete
SO 7 Complete
SO 30 Complete
SO 1000000 Complete
HBP 1 Complete
HBP 7 Complete
HBP 30 Complete
HBP 1000000 Complete
SF 1 Complete
SF 7 Complete
SF 30 Complete
SF 1000000 C

In [38]:
starting_pitcher_df = pitching_raw.loc[pitching_raw['GS'] == 1]

In [39]:
for col in starting_pitcher_df.columns[3:16]:
    for days in [30]: #1, 3, 7, 15, 30, 1000000
        gen_avg(starting_pitcher_df, col, days, 'sum')
        print('{0} {1} Complete'.format(col, str(days)))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


GS 30 Complete
W 30 Complete
L 30 Complete
SV 30 Complete
HLD 30 Complete
IP 30 Complete
TBF 30 Complete
H 30 Complete
R 30 Complete
ER 30 Complete
HR 30 Complete
BB 30 Complete
SO 30 Complete


In [37]:
for col in starting_pitcher_df.columns[3:16]:
    for days in [1, 7, 30, 1000000]: #1, 3, 7, 15, 30, 1000000
        gen_avg(starting_pitcher_df, col, days, 'sum')
        print('{0} {1} Complete'.format(col, str(days)))

NameError: name 'starting_pitcher_df' is not defined

## Add QS

In [40]:
def qs_gen(IP, ER):
    
    if IP >= 6 and ER <= 3:
        return 1
    else:
        return 0

In [41]:
## Add QS
starting_pitcher_df.loc[:, 'QS'] = starting_pitcher_df.loc[:, ['IP', 'ER']].apply(lambda x: qs_gen(*x), axis = 1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [42]:
for days in [30]: #1, 3, 7, 15, 30, 1000000
    gen_avg(starting_pitcher_df, 'QS', days, 'sum')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [43]:
for days in [1, 7, 30, 1000000]: #1, 3, 7, 15, 30, 1000000
    gen_avg(starting_pitcher_df, 'QS', days, 'sum')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [43]:
batter_raw.to_csv(r'..\Data_Pull\data_files\Simple_ML_files\bat_ml_one.csv')
starting_pitcher_df.to_csv(r'..\Data_Pull\data_files\Simple_ML_files\pitch_ml_one.csv')

## General Info Add

In [122]:
batter_gen = pd.merge(bat_all, gen_inf, how = 'left', on = 'pid')

In [123]:
batter_gen.loc[:, 'Age'] = (batter_gen.loc[:, 'Date'] - batter_gen.loc[:, 'Birthdate']).dt.days

Pitch needs pid added back

In [124]:
pitch_gen = pd.merge(starting_pitcher_df, gen_inf, how = 'left', on = 'pid')
pitch_gen.loc[:, 'Age'] = pitch_gen.loc[:, 'Date'] - pitch_gen.loc[:, 'Birthdate']

## Combine

In [125]:
starting_pitcher_df.columns = [str(col) + '_pitch' for col in starting_pitcher_df.columns]

In [126]:
batter_gen.drop_duplicates(inplace = True)
starting_pitcher_df.drop_duplicates(inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [127]:
batter_gen = batter_gen.loc[~((batter_gen['Teams'].isin(two_game_issue['Teams'].values)) & (batter_gen['Date'].isin(two_game_issue['Date'].values)))]

In [129]:
combine_df = pd.merge(batter_gen, starting_pitcher_df, how = 'left', left_on = ['Date', 'Teams', 'Opp_x'], right_on = ['Date_pitch', 'Teams_pitch', 'Team_pitch'])

In [65]:
bat_cols = [col for col in combine_df.columns if 'pitch' not in col]
pitch_cols = [col for col in combine_df.columns if 'pitch' in col]

In [230]:
combine_df.to_csv(r'..\Data_Pull\data_files\Simple_ML_files\combined_ml.csv')

In [66]:
combine_df.to_csv(r'..\Data_Pull\data_files\Simple_ML_files\combined_ml_163_only.csv')

In [130]:
combine_df.to_csv(r'..\Data_Pull\data_files\Simple_ML_files\combined_ml_30_only.csv')